## Deliverable 2: Project Proposal


### GENERAL INSTRUCTIONS:
- Written proposal within 500 words (about 1 page)   
- Identify the dataset they plan to work on  
- Identify the question they would like to answer using that dataset  
- Only one member of your team needs to submit. You must submit two files:
    - the source Jupyter notebook (.ipynb file)
    - the rendered final document (.html file)
 
### Introduction

1. Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal.
2. Clearly state the question you will try to answer with your project. 
    - Your question should involve one or more random variables of interest, spread across two or more categories that are interesting to compare. 
    - For example, you could consider the annual maxima river flow at two different locations along a river, or perhaps gender diversity at different universities. 
3. Of the response variables, identify one location parameter (mean, median, quantile, etc.) that would be useful in answering your question. 
4. Identify one scale parameter (standard deviation, inter-quartile range, etc.) that would be useful in answering your question. 
5. Justify your choices.
6. Identify and describe the dataset that will be used to answer the question.

#### Notes: 
- If it doesn’t make sense to infer a scale parameter, you can choose another parameter, or choose a second variable altogether. 
- Ultimately, we’re looking for a comprehensive inference analysis on one parameter spread across 2+ groups (with at least one hypothesis test), plus a bit more (such as an investigation on the variance, a quantile, or a different variable). 
- In total, you should use both bootstrapping and asymptotics somewhere in your report at least once each. 
- Also, your hypothesis test(s) need not be significant: it is perfectly fine to write a report claiming no significant findings (i.e. your p-value is large).
- Also, be sure to frame your question/objectives in terms of what is already known in the literature. 

### Preliminary Results

1. Demonstrate that the dataset can be read from the web into R.
2. Clean and wrangle your data into a tidy format.
3. Plot the relevant raw data, tailoring your plot in a way that addresses your question.
4. Compute estimates of the parameter you identified across your groups. 
5. Present this in a table. If relevant, include these estimates in your plot.

#### Notes:
Be sure to not print output that takes up a lot of screen space.

### Methods: Plan

1. Brief description of “the good things” about this report – specifically, in what ways is this report trustworthy?
2. Continue by explaining why the plot(s) and estimates that you produced are not enough to give to a stakeholder, and what you should provide in addition to address this gap. 
3. Make sure your plans include at least one hypothesis test and one confidence interval. If possible, compare both the bootstrapping and asymptotics methods.
4. Finish this section by reflecting on how your final report might play out:
    - What do you expect to find?
    - What impact could such findings have?
    - What future questions could this lead to?

### References

- At least two citations of literature relevant to the project. 
- The citation format is your choice – just be consistent. 
- Make sure to cite the source of your data as well.

### Potential Datasets
- https://archive.ics.uci.edu/datasets
- City of Vancouver data: https://opendata.vancouver.ca/pages/home/ (or other cities, like Ottawa, Toronto, etc.)
- Vancouver crime data: https://geodash.vpd.ca/opendata/
- BC data: https://data.gov.bc.ca/
- Water survey of Canada: https://www.canada.ca/en/environment-climate-change/services/water-overview/quantity/monitoring/survey.html
- Climate and Weather Data: https://climate.weather.gc.ca/